### 1. Install dependencies

- pandas → used for data handling and analysis
- scikit-learn → provides machine learning tools like LogisticRegression, CountVectorizer, etc.

%pip install pandas scikit-learn

### 2. Import required libraries

* `pandas` → for loading and managing dataset
* `train_test_split` → splits data into training and testing
* `CountVectorizer` → converts text to numerical features
* `LogisticRegression` → our ML model
* `accuracy_score`, `classification_report` → to evaluate the model

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report


### 3. Load the dataset

Download the dataset first from:
🔗 [https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset](https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset)

### 4. Read CSV

* Reads the CSV file into a pandas DataFrame.
* `encoding="latin-1"` avoids errors due to special characters.
* `df.head()` shows the first 5 rows of the dataset.


In [4]:
df = pd.read_csv("spam.csv", encoding="latin-1")
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


### 5. Clean the dataset
* We only need 2 columns:

  * `label` → whether message is “ham” or “spam”
  * `message` → actual SMS text

In [5]:
df = df[['v1', 'v2']]
df.columns = ['label', 'message']
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


### **6. Convert text labels to numbers**
* Machine learning models can’t understand text labels.
* We convert:

  * `ham` → 0
  * `spam` → 1
* New column `label_num` will contain numeric labels.

In [6]:
df['label_num'] = df['label'].map({'ham': 0, 'spam': 1})
df.head()

,label,message,label_num
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


### ✂️ 7. Split data into training and testing sets

* 80% of data → training
* 20% → testing
* `random_state=42` makes sure you get the same split every time.

#### 1️⃣ **80% of data → training**
The model needs to **learn patterns** — like which words appear more often in spam vs normal messages.
That learning happens from **training data**.

Example:
Let’s say you have 1000 SMS messages in your dataset.

* 800 messages (80%) are used to **train** the model — the model *studies* these.
* 200 messages (20%) are kept aside for **testing** — the model *never sees* these during learning.

Why?
Because if we test on the same data we trained on, the model might just memorize answers.
Testing on unseen data shows how well it *generalizes* to new examples.

🧠 Think of it like:

* > Training = studying for the exam
* > Testing = writing the actual exam

#### 2️⃣ **20% of data → testing**

The **testing set** checks how well the model performs on *unseen messages*.

* These messages were *not shown* to the model during training.
* We use them to measure **accuracy**, **precision**, **recall**, etc.

This gives us a realistic idea of how our model would perform on *real-world data* (like new messages coming tomorrow).


#### 3️⃣ **`random_state=42`**

Every time you split data randomly, Python shuffles your dataset in a random way.

That means:

```python
train_test_split(..., test_size=0.2)
```

will give you a *different* 80-20 split each time you run it.

To make results **consistent** and **reproducible**, we fix a “seed” using `random_state`.

💡 You can use *any number* (42 is just a common joke number used in data science 😄).
When `random_state=42`, the random shuffling happens in the *same pattern* every time —
so your results (accuracy, precision, etc.) stay the same whenever you re-run the notebook.

----

If you write:

```python
train_test_split(..., random_state=2)
```

you’ll get a particular random shuffle and split — let’s call it **Split A**.

If you change it to:

```python
train_test_split(..., random_state=3)
```

you’ll get a *different* shuffle and split — call it **Split B**.

But here’s the key point:

> Whenever you use the **same random_state value again**, you’ll get **the exact same split** every single time.


In [16]:
X_train, X_test, y_train, y_test = train_test_split(
    df['message'], df['label_num'], test_size=0.2, random_state=42    
)

### 8. Convert text to numerical vectors


* `CountVectorizer` converts each text into a “bag of words” numeric form.
* `fit_transform()` → learns the vocabulary and transforms training data
* `transform()` → converts test data using the same learned vocabulary


In [17]:
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

### 9. Train the Logistic Regression model

* `LogisticRegression` is a classification algorithm.
* The model learns from training data how words relate to being spam or not spam.

In [19]:
model = LogisticRegression()
model.fit(X_train_vec, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


### 10. Make predictions and evaluate

* `predict()` → predicts labels for test messages.
* `accuracy_score` → measures how many predictions are correct.
* `classification_report` → shows precision, recall, and F1-score.

In [20]:
y_pred = model.predict(X_test_vec)
print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("\n📊 Classification Report:\n", classification_report(y_test, y_pred))

✅ Accuracy: 0.9775784753363229

📊 Classification Report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99       965
           1       0.99      0.84      0.91       150

    accuracy                           0.98      1115
   macro avg       0.98      0.92      0.95      1115
weighted avg       0.98      0.98      0.98      1115



#### 🔹 1. **Precision**

> “Out of all predicted *spam*, how many were *actually spam*?”

$$
\text{Precision} = \frac{TP}{TP + FP}
$$

✅ High precision = your model **doesn’t lie much** (few false alarms).

**Example:**
- Model said 30 emails are spam.  
- Out of them, 27 were actually spam.  
  → Precision = 27 / 30 = **0.9 (90%)**

---

#### 🔹 2. **Recall**

> “Out of all *actual spam* emails, how many did the model catch?”

$$
\text{Recall} = \frac{TP}{TP + FN}
$$

✅ High recall = your model **catches most real spam**.

**Example:**
- There were 40 actual spam emails.  
- Model caught 27 of them.  
  → Recall = 27 / 40 = **0.675 (67.5%)**

---

#### 🔹 3. **F1-score**

> “A balance between precision and recall.”

$$
\text{F1} = 2 \times \frac{Precision \times Recall}{Precision + Recall}
$$

✅ F1 is high **only if both precision and recall are high** — it’s their *harmonic mean*.

**Example:**
$$
F1 = 2 \times \frac{0.9 \times 0.675}{0.9 + 0.675} \approx 0.77
$$


#### 🧩 What is **Support**?

**Support** = **number of actual samples** (from your test set) that belong to a particular class.

It tells you **how many true examples** of each label were present in `y_test`.


```
              precision    recall  f1-score   support
not spam          0.80      1.00      0.89        2
spam              1.00      0.67      0.80        3
accuracy                             0.87        5
macro avg         0.90      0.83      0.85        5
weighted avg      0.92      0.87      0.88        5
```

👉 The `support` column shows:

* `not spam` → **2 samples**
* `spam` → **3 samples**


### 11. Try with your own messages

* Converts your custom messages into vectors.
* Predicts whether each one is **spam or ham**.
* Prints human-readable results.

In [35]:
sample_msgs = [
    "Congratulations! You’ve won ₹10,000 cash. Claim your prize now!",
    "Get a free Netflix subscription — limited offer!",
    "Exclusive deal! 50% off on all electronics today only!",
    "WINNER!! As a valued network customer you have",
    "Had your mobile 11 months or more? U R entitle",
    "Your free ringtone is waiting to be collecte"
]

sample_vec = vectorizer.transform(sample_msgs)
predictions = model.predict(sample_vec)

for msg, pred in zip(sample_msgs, predictions):
    label = "SPAM 🚨" if pred == 1 else "HAM ✅"
    print(f"{msg} --> {label}")

Congratulations! You’ve won ₹10,000 cash. Claim your prize now! --> SPAM 🚨
Get a free Netflix subscription — limited offer! --> HAM ✅
Exclusive deal! 50% off on all electronics today only! --> HAM ✅
WINNER!! As a valued network customer you have --> HAM ✅
Had your mobile 11 months or more? U R entitle --> HAM ✅
Your free ringtone is waiting to be collecte --> HAM ✅
